In [12]:
#IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.spatial.transform import Rotation as R

import dataUtils as du

In [13]:
path_base = "Tracking/Gridsearch/"
a = 100 #acceleration in steps
hz = 30
samples = 3 #steps in gridsearch
stepsize = 20 #steps used to perform gridsearch
full_path = "" #custom path to gridsearch data
num_meta_rows = 6

#load all data
df = du.load_dataset(a,samples,stepsize,path_base,full_path)

# Change rotations to accurate coordinate system and make translations relative to frame 0
df = du.fixCoordinates(df)

#get valid frames (frames without wobeling/ocscilation)
start_offset =  65 #(frame before action)
wanted_frames = samples ** 6 #amount of frames to extract
frame_offset = 2*hz #time between moves
spacing = 10 #how many previous frames that are considered when testing for oscillation
t_eps = 0.5 #translational error
r_eps =0.5 #rotational error

df_valid, valid_idx = du.getValidFrames(df,wanted_frames,frame_offset,start_offset,spacing,t_eps,r_eps)

#load targets
comb_list = du.getGridsearchCableLengths(stepsize)
comb_list = du.getValidTargets(comb_list,valid_idx)
Y = np.array(comb_list)

seed = 42

TOTAL FRAMES: 729 (729)
INVALID FRAMES: 0, VALID FRAMES: 729


In [14]:
print(df_valid.max())
print(df_valid.min())

Frame      43805.000000
Time        1460.166667
X_rot          7.778208
Y_rot         10.491430
Z_rot          7.510272
X_trans        9.275269
Y_trans        5.542541
Z_trans       14.873856
dtype: float64
Frame      125.000000
Time         4.166667
X_rot       -6.316861
Y_rot      -10.544286
Z_rot       -7.188015
X_trans     -8.202880
Y_trans     -5.530762
Z_trans    -11.188171
dtype: float64


# Data representation and normalisation

In [15]:
#Regularize the input data
def raw(norm = True):
  col = ["X_trans", "Y_trans", "Z_trans", "X_rot", "Y_rot", "Z_rot"]
  X = df_valid[col].copy()
  if norm:
    for i in range(6):
      X[col[i]] = 2 * (X[col[i]] - X[col[i]].min()) / (X[col[i]].max() - X[col[i]].min()) - 1


  # Split data into train and test sets
  X_train, X_test, y_train, y_test = train_test_split(X.values, Y, test_size=0.15, random_state=seed)
  return X_train, X_test, y_train, y_test

In [16]:
# Marker normalize
def marker(norm = True):
  # min-max scale translations to be in [-1,1]
  col = [ "X_trans", "Y_trans", "Z_trans"]
  X_temp = df_valid.copy()
  for c in col:
    X_temp[c] = 2 * (X_temp[c] - X_temp[c].min()) / (X_temp[c].max() - X_temp[c].min()) - 1
  # Selecting features from column index 2 to 7 (exclusive of 8)
  X =X_temp.iloc[:, 2:8].values
  rotations = X[:, :3]
  translations = X[:, 3:]

  # Define the positions of 4 points spaced around the original pivot point
  # These positions are relative to the pivot point before rotation
  point_offsets = np.array([[1, 0, 1],  # Point 1
                            [0, 1, -1],  # Point 2
                            [-1, -1, 0], # Point 3
                            [-1, 0, -1]])# Point 4

  # Convert Euler angles to rotation matrices
  rot_matrices = R.from_euler('xyz', rotations, degrees=True).as_matrix()

  # Apply rotation to the point offsets
  rotated_points = np.matmul(rot_matrices, point_offsets.transpose())
  rotated_points = np.transpose(rotated_points, axes=(0, 2, 1))


  for i in range(rotated_points.shape[0]):
    for point in rotated_points[i]:
      point[0] += translations[i,0]
      point[1] += translations[i,1]
      point[2] += translations[i,2]

  X = rotated_points.reshape(-1,12)
  if norm:
    # 12 features, xyz xyz xyz xyz
    min_xs, max_xs = min([np.min(X[:,0]),np.min(X[:,3]),np.min(X[:,6]),np.min(X[:,9])]), max([np.max(X[:,0]),np.max(X[:,3]),np.max(X[:,6]),np.max(X[:,9])])
    min_ys, max_ys = min([np.min(X[:,1]),np.min(X[:,4]),np.min(X[:,7]),np.min(X[:,10])]), max([np.max(X[:,1]),np.max(X[:,4]),np.max(X[:,7]),np.max(X[:,10])])
    min_zs, max_zs = min([np.min(X[:,2]),np.min(X[:,5]),np.min(X[:,8]),np.min(X[:,11])]), max([np.max(X[:,2]),np.max(X[:,5]),np.max(X[:,8]),np.max(X[:,11])])

    #normalize all vectors based on their max/min x,y or z value
    for i in range(X.shape[0]):
      for x in [0,3,6,9]:
        X[i,x] = 2*((X[i,x]-min_xs)/(max_xs-min_xs)) - 1
      for y in [1,4,7,10]:
        X[i,y] = 2*((X[i,y]-min_ys)/(max_ys-min_ys)) -1
      for z in [2,5,8,11]:
        X[i,z] = 2*((X[i,z]-min_zs)/(max_zs-min_zs)) -1


  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=seed)
  return X_train, X_test, y_train, y_test

In [17]:
def marker_split(norm = True):
  # min-max scale translations to be in [-1,1]
  col = [ "X_trans", "Y_trans", "Z_trans"]
  X_norm = df_valid.copy()
  if norm:
    for c in col:
      X_norm[c] = 2 * (X_norm[c] - X_norm[c].min()) / (X_norm[c].max() - X_norm[c].min()) - 1
  # Selecting features from column index 2 to 7 (exclusive of 8)
  X =X_norm.iloc[:, 2:8].values
  rotations = X[:, :3]
  translations = X[:, 3:]
  # Define the positions of 4 points spaced around the original pivot point
  # These positions are relative to the pivot point before rotation
  point_offsets = np.array([[1, 0, 1],  # Point 1
                            [0, 1, -1],  # Point 2
                            [-1, -1, 0], # Point 3
                            [-1, 0, -1]])# Point 4

  # Convert Euler angles to rotation matrices
  rot_matrices = R.from_euler('xyz', rotations, degrees=True).as_matrix()

  # Apply rotation to the point offsets
  rotated_points = np.matmul(rot_matrices, point_offsets.transpose())
  rotated_points = np.transpose(rotated_points, axes=(0, 2, 1))
  rotated_points = rotated_points.reshape(-1,12)
  X =np.concatenate((rotated_points,translations),axis = 1) #makes translations/rotation be separate

  if norm:
  # vector normalize each "rotation" marker
    for i in range(X.shape[0]):
      for x in [0,3,6,9]:
        X[i,x] = X[i,x] / np.sqrt(2)
      for y in [1,4,7,10]:
        X[i,y] = X[i,y] / np.sqrt(2)
      for z in [2,5,8,11]:
        X[i,z] = X[i,z] / np.sqrt(2)

  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=seed)
  return X_train, X_test, y_train, y_test

In [18]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = raw(norm = True)

# Random Forest Regression

In [19]:
# Initialize the RandomForestRegressor
random_forest = RandomForestRegressor(n_estimators=100, random_state=seed)

# Specify the number of folds for cross-validation
k_folds = 5

# Initialize the KFold cross-validator
kf = KFold(n_splits=k_folds, shuffle=True, random_state=seed)

# Initialize list to store validation MSE scores
validation_mse_scores = []

# Iterate over each fold
for train_index, val_index in kf.split(X_train):
    # Split the data into training and validation sets for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    
    # Fit the model on the training data
    random_forest.fit(X_train_fold, y_train_fold)
    
    # Predict on the validation set
    y_pred = random_forest.predict(X_val_fold)
    
    # Compute MSE for this fold
    mse = mean_squared_error(y_val_fold, y_pred)
    validation_mse_scores.append(mse)

# Print the validation MSE scores
print("Validation MSE scores:", validation_mse_scores)
print("Mean Validation MSE:", np.mean(validation_mse_scores))
print("Std Validation MSE:", np.std(validation_mse_scores))

Validation MSE scores: [52.08849462365591, 55.9089247311828, 51.906129032258065, 54.14306451612904, 50.76130081300814]
Mean Validation MSE: 52.96158274324679
Std Validation MSE: 1.833042822557558


# Feedforward Neural Network

In [20]:
class FeedForwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Define input and output sizes
input_size = 6 # 6 for raw, 12 for marker, 15 for marker_split
output_size = 6
hidden_size = 32
batch_size = 4
lr = 0.001
num_epochs = 500

# Create the model
model = FeedForwardNN(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed

In [21]:
#SETUP DATALOADERS
from torch.utils.data import TensorDataset, DataLoader

def create_data_loaders(train_index, val_index):
    X_train_fold,y_train_fold = torch.tensor(X_train[train_index], dtype=torch.float32), torch.tensor(y_train[train_index], dtype=torch.float32)
    X_val_fold,y_val_fold = torch.tensor(X_train[val_index], dtype=torch.float32), torch.tensor(y_train[val_index], dtype=torch.float32)
    train_dataset = TensorDataset(X_train_fold,y_train_fold)
    val_dataset = TensorDataset(X_val_fold,y_val_fold)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader

In [ ]:
#TRAINING
# Lists to store training and validation losses
train_losses = []
val_losses = []
i = 0
criterion = nn.MSELoss()
for train_index, val_index in kf.split(X_train):
    # Initialize early stopping parameters
    # best_val_loss = np.Inf
    # patience = 15
    # counter = 0

    # Create the model
    model = FeedForwardNN(input_size, hidden_size, output_size)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    train_losses_fold = []
    val_losses_fold = []
    train_loader,val_loader = create_data_loaders(train_index, val_index)

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_train_loss = 0.0
        
        # Training loop
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item()
        
        # Compute average training loss for the epoch
        average_train_loss = running_train_loss / len(train_loader)
        train_losses_fold.append(average_train_loss)
        
        # Validation loop
        model.eval()  # Set the model to evaluation mode
        running_val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                running_val_loss += loss.item()
        
        # Compute average validation loss for the epoch
        average_val_loss = running_val_loss / len(val_loader)
        val_losses_fold.append(average_val_loss)

         # Check for improvement in validation loss
        # if running_val_loss < best_val_loss:
        #     best_val_loss = running_val_loss
        #     counter = 0
        #     # Save the best model state if needed
        #     # torch.save(model.state_dict(), 'best_model.pt')
        # else:
        #     counter += 1
        #     # If no improvement for patience epochs, stop training
        #     if counter >= patience:
        #         print(f'Early stopping at epoch {epoch}.')
        #         break
        
        # Print training and validation loss for the epoch
        print(f"Fold: {i}, Epoch [{epoch+1}/{num_epochs}], Train Loss: {average_train_loss:.4f}, Val Loss: {average_val_loss:.4f}", end = "\r")
    i += 1
    train_losses.append(train_losses_fold)
    val_losses.append(val_losses_fold)

# Plot the training and validation losses
plt.figure()
for i in range(k_folds):
    plt.plot(train_losses[i], label='Train Loss')
    plt.plot(val_losses[i], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
# SAVE LOSS
# np.save("data/20_new/3hid_64_neu_500epoch_p001lr_norm_trainloss",np.array(train_losses))
# np.save("data/20_new/3hid_64_neu_500epoch_p001lr_norm_valloss",np.array(val_losses))

# Train model on complete training data set (no validation), compared with test set

In [ ]:
# Lists to store training and validation losses

# Create the model
model = FeedForwardNN(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed

train_data_tensor = torch.tensor(X_train, dtype=torch.float32)
targets_tensor = torch.tensor(y_train, dtype=torch.float32)
val_data_tensor = torch.tensor(X_test, dtype=torch.float32)
val_targets_tensor = torch.tensor(y_test, dtype=torch.float32)
# Combine the input data and target values into a TensorDataset
train_dataset = TensorDataset(train_data_tensor, targets_tensor)
test_dataset = TensorDataset(val_data_tensor, val_targets_tensor)

# Create a DataLoader for batch processing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_train_loss = 0.0
    
    # Training loop
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()
    
    # Compute average training loss for the epoch
    average_train_loss = running_train_loss / len(train_loader)
    train_losses.append(average_train_loss)
    
    # Validation loop
    model.eval()  # Set the model to evaluation mode
    running_val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            running_val_loss += loss.item()
    
    # Compute average validation loss for the epoch
    average_val_loss = running_val_loss / len(val_loader)
    val_losses.append(average_val_loss)
    
    # Print training and validation loss for the epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {average_train_loss:.4f}, Val Loss: {average_val_loss:.4f}", end = "\r")

KeyboardInterrupt: 

In [ ]:
# #SAVE MODEL
#model_name = "3hid_32_neu_500epoch_p001lr_norm-11_rev20"
#torch.save(model, "models/"+ model_name + ".pth")